# Crop Yield Prediction

## Data Cleaning

In [ ]:
#import necessarty libraries
import numpy as np 
import pandas as pd 
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

In [ ]:
#import the data set
yield_df = pd.read_csv('/Users/chirag/Downloads/yield_df.csv')
yield_df = yield_df.drop(yield_df.columns[0], axis=1)
yield_df.head()

In [ ]:
#a top-down exploratory data analysis of the yield dataset that covers a multitude of key statistics
profile = ProfileReport(yield_df, title = "Exploratory Data Analysis of Yield Data")
profile.to_notebook_iframe()

<p> According to the correlation heatmap and the corresponding table, there are no correlations between any of the variables in the yield dataframe. We also don't have any missing values in any our columns. Nice! In addition, we have 2101 duplicate rows which accounts for approximately 7.4% of all the rows. Some of the duplicate even show up 4 times, indicating there is a combination of factors which allow for the same results to show up time after time. 
</p>

In [ ]:
#a more concise look at key statistics
yield_df.describe()

In [ ]:
#order the 101 countries by the 10 highest yield productions
yield_df.groupby(['Area'],sort=True)['hg/ha_yield'].sum().nlargest(10)

In [ ]:
#group the number of items in each area by the yield
yield_df.groupby(['Item','Area'],sort=True)['hg/ha_yield'].sum().nlargest(10)

## Preprocess Data for Modelling

<p> I chose to employ one-hot encoding to convert our catagorical variables (Area, Item) into numerical data so they can be used in our machine-learning model of choice. The encoding creates a binary column for each category and returns a matrix with the results.
</p>

In [ ]:
yield_df_onehot = pd.get_dummies(yield_df, columns=['Area',"Item"], prefix = ['Country',"Item"])
features=yield_df_onehot.loc[:, yield_df_onehot.columns != 'hg/ha_yield']
label=yield_df['hg/ha_yield']
features.head()

In [ ]:
#the year should not have much of an effect on the model so I chose to drop it
features = features.drop(['Year'], axis=1)
features.head()

<p> Our dataset contains features that highly vary in magnitudes, units, and range. Features with higher magnitudes will most likely have a heavier weight in our distance calculations when running our machine learning models (Random Forest, Gradient Boosting, and Decision Tree). Here, I ensure that all of the features are of the same magnitude via scaling. 
</p>

In [ ]:
scaler=MinMaxScaler()
features=scaler.fit_transform(features) 
features

<p> To train our machine learning algorithms, let's split the dataset into two (training and testing datsets). Since we want the training set to have as much data as possible, a 70/30 or 80/20 split in favor of the training set is desireable. 
</p>

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(features, label, test_size=0.2, random_state=42)

## Compare Models

In [ ]:
def compare_models(model):
    model_name = model.__class__.__name__
    fit=model.fit(train_data,train_labels)
    y_pred=fit.predict(test_data)
    r2 = r2_score(test_labels,y_pred)
    msq = mean_squared_error(test_labels,y_pred, squared = False) #setting the 'argument' to false return the RMSE
    return([model_name,r2,msq])

<p> First, we retrieve the name of the model by calling the '__class__.__name__' attribute, which returns the name of the class of the input model object. The function then fits the model object to a training dataset 'train_data' with corresponding labels 'train_labels', using the 'fit()' method of the model. Once the model is fit, the function generates predictions for a test dataset 'test_data' using the predict() method, and calculates the R-squared (coefficient of determination) score between the predicted and actual labels using the 'r2_score()' function. Finally, the function returns a list containing the name of the model and the R-squared score as computed by 'r2_score()'.
</p>

In [ ]:
models = [
     GradientBoostingRegressor(n_estimators=200, max_depth=3, random_state=0),
     RandomForestRegressor(n_estimators=200, max_depth=3, random_state=0),
    svm.SVR(),
   DecisionTreeRegressor()
]


In [ ]:
#trains each machine learning model in the models list using the compare_models function
model_train=list(map(compare_models,models)) 

In [ ]:
print(*model_train, sep = "\n")


<p> In our result above, we see the name of the algorithm next to its respective R^2 score, which is a statistical measure that represents the proportion of variance in the dependent variable (i.e., the output or target variable in a supervised learning problem) that is explained by the independent variables. Here, we use it as a measure of how well the model fits the data. The R^2 score ranges from 0 to 1, with a higher value indicating a better fit of the model to the data. A score of 1 indicates that the model perfectly fits the data, while a score of 0 indicates that the model does not explain any of the variance in the dependent variable. Using these guidelines, we can see that the Decision Tree Regressor model fits the data much better than any of our other models.
</p>

<p> To the right of our R^2 score lies our RMSE (Root Mean Squared Error) for each model. It measures the difference between the predicted values and the actual values of the target variable and is expressed in the same units as our target variable. A lower RMSE indicates that the model has better predictive power and is more accurate. The motivation behind calculating both R^2 and RMSE was to understand the performance of each model in a more comprehensive manner and to better justify choosing one model over the others. As such, the Decision Tree Regressor Model has the lowest RMSE score, making it the most ideal model for our data.
</p>

In [ ]:
yield_df_onehot = yield_df_onehot.drop(['Year'], axis=1)
yield_df_onehot.head()

In [ ]:
test_df=pd.DataFrame(test_data,columns=yield_df_onehot.loc[:, yield_df_onehot.columns != 'hg/ha_yield'].columns) 

cntry=test_df[[col for col in test_df.columns if 'Country' in col]].stack()[test_df[[col for col in test_df.columns if 'Country' in col]].stack()>0]
cntrylist=list(pd.DataFrame(cntry).index.get_level_values(1))
countries=[i.split("_")[1] for i in cntrylist]
itm=test_df[[col for col in test_df.columns if 'Item' in col]].stack()[test_df[[col for col in test_df.columns if 'Item' in col]].stack()>0]
itmlist=list(pd.DataFrame(itm).index.get_level_values(1))
items=[i.split("_")[1] for i in itmlist]

test_df.head()

In [ ]:
test_df.drop([col for col in test_df.columns if 'Item' in col],axis=1,inplace=True)
test_df.drop([col for col in test_df.columns if 'Country' in col],axis=1,inplace=True)

test_df['Country']=countries
test_df['Item']=items

test_df.head()

In [ ]:
clf=DecisionTreeRegressor()
model=clf.fit(train_data,train_labels)

test_df["yield_predicted"]= model.predict(test_data)
test_df["yield_actual"]=pd.DataFrame(test_labels)["hg/ha_yield"].tolist()
test_group=test_df.groupby("Item")

<p> Here, we create an instance of a decision tree regressor model. We then fit the model to the training data and labels, and stores the resulting trained model in the 'model' variable. Afterwards we predict the yield for the test dataset using the trained model and stores the predicted yield values in a new column called "yield_predicted" in the 'test_df' dataframe. In order to compare the predicted values to the actual values, we extract the actual yield values from the test labels dataframe and store them in a new column called "yield_actual" in the test_df dataframe. The last line groups the test_df dataframe by the "Item" column.
</p>

In [ ]:
test_group.apply(lambda x: r2_score(x.yield_actual,x.yield_predicted))

<p> This calculates the R-squared score for each group (i.e., each crop) using the predicted and actual yield values, and returns a pandas Series object with the R-squared score for each group.
</p>

In [ ]:
test_group.apply(lambda x: mean_squared_error(x.yield_actual,x.yield_predicted, squared = False))

<p> This computes the root mean squared error (RMSE) for each group in the test_df dataframe, where the groups are defined by the "Item" column. 
</p>

In [ ]:
import plotly.express as px

fig = px.scatter(test_df, x="yield_actual", y="yield_predicted")
fig.update_traces(marker=dict(color="Black", size=3, opacity=0.5))
fig.update_layout(
    title="Actual vs Predicted",
    xaxis_title="Actual",
    yaxis_title="Predicted",
    font=dict(size=12),
    margin=dict(l=50, r=50, b=50, t=50, pad=4)
)
fig.show()


In [ ]:
def adjusted_r_squared(y, yhat, X):
    n = len(y)
    k = X.shape[1]
    r2 = r2_score(y, yhat)
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - k - 1)
    return adj_r2

test_group.apply(lambda x: adjusted_r_squared(x.yield_actual,x.yield_predicted,x))

<p> The adjusted R-squared is a modified version of R-squared that adjusts for the number of input features used in the model, and it provides a more accurate measure of the model's goodness of fit.
</p>

## Model Results

In [ ]:
# Get the feature importances of the model
imp = model.feature_importances_
# Get the names of the features, excluding the target column "hg/ha_yield"
names = yield_df_onehot.columns[yield_df_onehot.columns!="hg/ha_yield"]
# Create a dictionary with the feature importances and names
varimp = {'imp': imp, 'names': names}

### Figure A - "Feature Importance"

In [ ]:
df=pd.DataFrame.from_dict(varimp)
df.sort_values(ascending=False,by=["imp"],inplace=True)
df=df.dropna()

fig = px.bar(df, x='imp', y='names', orientation='h', color='imp',
             color_continuous_scale='RdBu', title='Feature Importance')


fig.update_layout(xaxis_title='Importance', yaxis_title='Features', 
                  xaxis_tickformat=',.0%', height=600, margin=dict(l=200, r=50, t=50, b=50))

fig.show()


### Figure B - "Top 7 Most Important Factors Affecting Crp Yield"

In [ ]:
fig = px.bar(df.sort_values('imp', ascending = False).head(7),
             x='imp',
             y='names',
             orientation='h',
             color='imp',
             color_continuous_scale='RdBu_r')
fig.update_layout(title='Top 7 Most Important Factors Affecting Crop Yield')
fig.show()

### Figure C - "Yield for Each Item"

In [ ]:
fig = px.box(yield_df, x='Item', y='hg/ha_yield', color='Item', color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_layout(title='Yield for Each Item', xaxis_title='Item', yaxis_title='Yield (hg/ha)')
fig.show()

## Results

<p> From figure B, we see that crops being potatoes have the highest importance in the decision making for the model, especially since that it has the highest yield out of all the crops regardless of any other factors. The tonnage of pesticides beat out the average mm of rainfall per year as the most important external factor to crop growth. This was surprising because pesticides actually have negative effects on crop physiology―especially on photosynthesis―leading to a potential decrease in both the growth and the yield of crops. But it is no surpise that crops grown in India have the largest yield since India does have the largest crop sum in the entire dataset.
</p>